In [1]:
import os
import sys
import tensorflow as tf
import numpy      as np

In [2]:
InputFile      = '../input/O3_UMN/'


WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
SurQCTFldr     = WORKSPACE_PATH + '/SurQCT/surqct/'

print("\n======================================================================================================================================")
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))


 TensorFlow version: 2.4.1
 Eager execution: True


In [6]:
##==============================================================================================================
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
# from Reading import read_data, read_losseshistory
sys.path.insert(0, SurQCTFldr  + '/src/Plotting/')
from Plotting import plot_losseshistory
# sys.path.insert(0, SurQCTFldr  + '/src/Saving/')
# from Saving import save_parameters, save_data


InputFile = '../input/O3_UMN_KExcit/'
print("[SurQCT]:   Calling SurQCT with Input File = ", InputFile)
sys.path.insert(0, InputFile)
##--------------------------------------------------------------------------------------------------------------


##==============================================================================================================
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)
##--------------------------------------------------------------------------------------------------------------


##==============================================================================================================
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model
# if (InputData.ApproxModel == 'FNN'):
#     from Model import FNN

sys.path.insert(0, SurQCTFldr  + '/src/RatesType/' + InputData.RatesType + '/')
from RatesType import generate_data
# Generating Data
InputData, TrainData, TestData, AllData, ExtraData = generate_data(InputData)
##--------------------------------------------------------------------------------------------------------------


PathToFldr = InputData.PathToRunFld
try:
    os.makedirs(PathToFldr)
except OSError as e:
    pass


[SurQCT]: Loading Modules and Functions ...
[SurQCT]:   Calling SurQCT with Input File =  ../input/O3_UMN_KExcit/

[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...

[SurQCT]: Loading Final Modules ... 


In [7]:
### Initializing the Surrogate Model
NN = model(InputData, TrainData)

### Training the Surrogate Model
if (InputData.TrainIntFlg >= 1):
    History = NN.train(InputData)


[ProPDE]: Loading Data ... 
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
tf.split (TFOpLambda)           [(None, 10), (None,  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 10)           21          tf.split[0][0]                   
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 10)           21          tf.split[0][1]                   
_________________________________________________________________

6/6 [==============================] - 0s 5ms/step - loss: 92.3304 - val_loss: 91.9537
Epoch 53/100
6/6 [==============================] - 0s 5ms/step - loss: 91.9580 - val_loss: 91.5794
Epoch 54/100
6/6 [==============================] - 0s 5ms/step - loss: 91.5718 - val_loss: 91.1968
Epoch 55/100
6/6 [==============================] - 0s 6ms/step - loss: 91.1504 - val_loss: 90.8050
Epoch 56/100
6/6 [==============================] - 0s 5ms/step - loss: 90.7999 - val_loss: 90.4051
Epoch 57/100
6/6 [==============================] - 0s 5ms/step - loss: 90.4118 - val_loss: 89.9969
Epoch 58/100
6/6 [==============================] - 0s 6ms/step - loss: 89.9681 - val_loss: 89.5797
Epoch 59/100
6/6 [==============================] - 0s 6ms/step - loss: 89.5583 - val_loss: 89.1540
Epoch 60/100
6/6 [==============================] - 0s 5ms/step - loss: 89.1514 - val_loss: 88.7203
Epoch 61/100
6/6 [==============================] - 0s 5ms/step - loss: 88.7112 - val_loss: 88.2782
Epoch 62/100


In [8]:
%matplotlib inline

### Plotting the Losses History
PathToFldr = InputData.PathToFigFld
try:
    os.makedirs(PathToFldr)
except OSError as e:
    pass
plot_losseshistory(InputData, History)

In [9]:
### Generating Test Results
if (InputData.TestIntFlg >= 1):
    xAll      = AllData[0]
    yAll      = AllData[1]
    xExtra    = ExtraData
    ChooseVec = list(xAll.columns)
    ChooseVec.remove('Idx_i')


    for TTran in InputData.TTranVecTest:
        MaskVec   = (xAll['TTran_i'] == TTran)
        xPred     = xAll.loc[MaskVec]
        yPred     = NN.Model.predict(xPred[ChooseVec])
        yData     = yAll.loc[MaskVec]
        

        TrainingVar = 'log10(' + InputData.RatesType + ')'
        NLevels     = len(yPred)
        Indx        = np.arange(NLevels)+1

        Idxs = xPred.Idx_i.unique()
        
        for iIdx in Idxs:
            Idxs = xPred.Idx_i.unique()
            kIdx = xPred.index[xPred['Idx_i'] == iIdx].tolist()

            fig = plt.figure()
            plt.xlabel('Level Index')
            plt.ylabel('$K_i^Excit$ [$cm^3$/s]')
            if (len(yData)>0):
                plt.scatter(xPred['Idx_j'][kIdx], yData[TrainingVar][kIdx], c='k', marker='+', linewidths =0.5, label='Data')
            plt.scatter(xPred['Idx_j'][kIdx], yPred[kIdx], c='r', marker='x', linewidths =0.5, label='Predicted')
            plt.legend()
            plt.grid()
            plt.title('Excitation Rates at T = ' + str(int(TTran)) + 'K')
            FigPath = InputData.PathToFigFld + '/TestCase_' + str(int(TTran)) + 'K.png'
            fig.savefig(FigPath, dpi=600)
            plt.show()
            #plt.close()

#         ### Plotting Test Results
#         if (InputData.PlotIntFlg >= 1):
#             plot_prediction(InputData, xPred, yData, yPred, TTran)


#     for TTran in InputData.TTranVecExtra:
#         MaskVec = (xExtra['TTran_i'] == TTran)
#         xPred   = xExtra.loc[MaskVec]
#         yPred   = NN.Model.predict(xPred[ChooseVec])
#         yData   = []

#         ### Plotting Test Results
#         if (InputData.PlotIntFlg >= 1):
#             plot_prediction(InputData, xPred, yData, yPred, TTran)

In [20]:
xPred['Idx_i']

1000

In [19]:
kIdx

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


array([-2.36883642e+00, -4.17454492e+00, -1.85874705e-01, -3.64794962e+00,
       -5.07458024e-01, -4.35603163e+00, -1.08842039e-05, -1.27518559e+00,
       -1.15318906e+00, -2.98601882e+00])